| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) |
| ----------------------------------- | -------------------------------------------- | ------------- | --------------------------------------- | --------------------------------- | --------------------------------- | --------------------------------------- | ------------------------------- | ----------------------------- |

# Transforming Geochemical Data

In [1]:
import pyrolite.geochem
import pandas as pd
import numpy as np
from pyrolite.util.synthetic import normal_frame

pd.set_option("precision", 3)  # smaller graphical outputs
 
df = normal_frame(columns=['CaO', 'MgO', 'SiO2', 'FeO','Na2O', 'Ni', 'Ti', 'La', 'Lu', 'Te']) * 100
df[['Ni', 'Ti', 'La', 'Lu', 'Te']] *= 10
df['Sr87/Sr86'] = 0.0700  / 0.0986 + np.random.randn(df.index.size) * 0.0001
df

,CaO,MgO,SiO2,FeO,Na2O,Ni,Ti,La,Lu,Te,Sr87/Sr86
0,3.124,8.667,39.832,9.058,2.857,33.555,26.279,93.351,89.418,122.013,0.71
1,3.036,9.570,38.025,8.733,2.820,34.957,26.422,101.125,91.235,124.431,0.71
2,3.035,9.404,38.031,8.984,3.121,34.634,25.648,99.232,95.920,118.824,0.71
3,2.943,9.184,38.678,8.280,2.741,33.207,23.610,98.711,93.732,132.488,0.71
4,4.125,10.689,33.614,9.655,2.907,42.121,24.577,104.685,90.823,127.894,0.71
5,3.388,10.234,36.724,9.090,2.804,37.510,23.745,104.747,94.834,116.766,0.71
6,3.158,8.910,37.747,8.554,2.875,34.649,26.383,97.306,96.443,132.788,0.71
7,3.101,9.680,37.742,8.755,3.113,34.845,25.468,101.144,91.133,123.494,0.71
8,3.155,9.130,37.580,8.378,2.725,35.396,24.593,101.429,93.099,135.813,0.71
9,3.775,10.173,35.258,9.130,2.787,39.867,24.241,104.679,90.269,129.710,0.71


----
### Using Indexers, Scaling

You can also use these indexers for assignment, where the dimensionality of the dataset doesn't change. While you can transform elements and oxide abundnace units easily when you remember the relative scales, `pyrolite` provides some functions such that you don't have to rely on your memory. Here we create a copy of the dataframe and within it revert the change we made above - so these should be the orignal ppm values. This method provides an easy way to explicitly declare your intention when changing units - and makes sure the relative scales are correct!

In [2]:
from pyrolite.util.units import scale

els = df.pyrochem.elements
els.pyrochem.elements *= scale('ppm', 'wt%')

C:\Users\WIL9DH\AppData\Local\Continuum\anaconda3\envs\pyrolite\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\WIL9DH\AppData\Local\Continuum\anaconda3\envs\pyrolite\lib\site-packages\pandas\core\indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [3]:
df.pyrochem.elements, els.pyrochem.elements

(       Ni      Ti       La      Lu       Te
 0  33.555  26.279   93.351  89.418  122.013
 1  34.957  26.422  101.125  91.235  124.431
 2  34.634  25.648   99.232  95.920  118.824
 3  33.207  23.610   98.711  93.732  132.488
 4  42.121  24.577  104.685  90.823  127.894
 5  37.510  23.745  104.747  94.834  116.766
 6  34.649  26.383   97.306  96.443  132.788
 7  34.845  25.468  101.144  91.133  123.494
 8  35.396  24.593  101.429  93.099  135.813
 9  39.867  24.241  104.679  90.269  129.710,
       Ni     Ti     La     Lu     Te
 0  0.003  0.003  0.009  0.009  0.012
 1  0.003  0.003  0.010  0.009  0.012
 2  0.003  0.003  0.010  0.010  0.012
 3  0.003  0.002  0.010  0.009  0.013
 4  0.004  0.002  0.010  0.009  0.013
 5  0.004  0.002  0.010  0.009  0.012
 6  0.003  0.003  0.010  0.010  0.013
 7  0.003  0.003  0.010  0.009  0.012
 8  0.004  0.002  0.010  0.009  0.014
 9  0.004  0.002  0.010  0.009  0.013)

---
### Converting Chemical Components 

`pyrolite` provides some straightfoward methods to calcuate element-oxide conversions (e.g. to transform Ti abundance to TiO2 abudnance), assuming that the system is open to oxygen (i.e. in this case the extra oxygen will be added to the composition). This interface also allows the user to quickly add ratios and specify redox pairs at the same time. For example, we can transform a copy of our dataframe to include extra ratios and change some of our oxide components to elements:

In [4]:
df.pyrochem.convert_chemistry(
    to=["MgO", "SiO2", "FeO", "Ca", "Te", "Na", "Na/Te", "MgO/SiO2"]
)

,Sr87/Sr86,MgO,SiO2,FeO,Ca,Te,Na,Na/Te,MgO/SiO2
0,0.71,8.667,39.832,9.058,2.233,122.013,2.120,0.017,0.218
1,0.71,9.570,38.025,8.733,2.170,124.431,2.092,0.017,0.252
2,0.71,9.404,38.031,8.984,2.169,118.824,2.316,0.019,0.247
3,0.71,9.184,38.678,8.280,2.103,132.488,2.033,0.015,0.237
4,0.71,10.689,33.614,9.655,2.948,127.894,2.157,0.017,0.318
5,0.71,10.234,36.724,9.090,2.422,116.766,2.080,0.018,0.279
6,0.71,8.910,37.747,8.554,2.257,132.788,2.133,0.016,0.236
7,0.71,9.680,37.742,8.755,2.216,123.494,2.310,0.019,0.256
8,0.71,9.130,37.580,8.378,2.255,135.813,2.022,0.015,0.243
9,0.71,10.173,35.258,9.130,2.698,129.710,2.068,0.016,0.289


In a similar way, we can also specify the molar speciation for redox species (so far just iron; others could be incorporated if they'll be useful). Here we adjust the total iron within our compositions (currently specified as FeO) to have a $Fe^{2+}/Fe^{3+}$ ratio of 9:1 (roughly what you might expect from a normal mantle-derived magma):

In [5]:
df.pyrochem.convert_chemistry(to=[{"FeO": 0.9, "Fe2O3": 0.1}])

,Sr87/Sr86,FeO,Fe2O3
0,0.71,8.152,1.007
1,0.71,7.859,0.971
2,0.71,8.085,0.998
3,0.71,7.452,0.920
4,0.71,8.690,1.073
5,0.71,8.181,1.010
6,0.71,7.699,0.951
7,0.71,7.880,0.973
8,0.71,7.540,0.931
9,0.71,8.217,1.015


| [**Overview**](./00_overview.ipynb) | [Getting Started](./01_jupyter_python.ipynb) | **Examples:** | [Access](./02_accessing_indexing.ipynb) | [Transform](./03_transform.ipynb) | [Plotting](./04_simple_vis.ipynb) | [Norm-Spiders](./05_norm_spiders.ipynb) | [Minerals](./06_minerals.ipynb) | [lambdas](./07_lambdas.ipynb) |
| ----------------------------------- | -------------------------------------------- | ------------- | --------------------------------------- | --------------------------------- | --------------------------------- | --------------------------------------- | ------------------------------- | ----------------------------- |